In [ ]:
import pickle
import pandas as pd
import numpy as np
import pymongo
import json
import pprint
import requests
import time
from pymongo.errors import BulkWriteError

## Use the entries in our database to retrieve the RSS feeds through the iTunes API
With the first batch of data in the database we can start retrieving the RSS feeds from Apple. Later we will use the RSS info to get the epsiodes of each podcast. However, the first thing we do is to add another index to the data base. The entries in the podcast collection received a unique identifier when they were inserted, but we want to also have a unique entry for every iTunes ID. 

In [ ]:
# connect to cloud data base
mongo_db_login_string = pickle.load( open( 'podcast-scraping/mongo_db_login_string.pickle', "rb" ) )

In [ ]:
client = pymongo.MongoClient(mongo_db_login_string)

In [ ]:
db = client.podcasts

In [ ]:
# try to get podcast collection
collection = db.podcast

In [ ]:
# add index to the podcast collection using the itunes_id entry
result = db.podcast.create_index([('itunes_id', pymongo.ASCENDING)], unique=True)

In [ ]:
query_results = collection.find( { "rss_feed":None } ).limit(20)

In [ ]:
id_list = [item['itunes_id'] for item in query_results]

In [ ]:
id_list

In [ ]:
rss_dict = {}
for id in id_list:
    
    itunes_search_link = 'https://itunes.apple.com/lookup?id=' + id.replace('id', '')
    
    search_request = requests.get(itunes_search_link).json()
    
    # insert feed into dict
    rss_dict[id] = search_request['results'][0]['feedUrl']


In [ ]:
search_request = requests.get(itunes_search_link).json()

In [ ]:
search_request

In [ ]:
for key in rss_dict:
    
    result = collection.find_one_and_update(
        {'itunes_id' : key},
        {"$set":
            {'rss_feed': rss_dict[key]}
        },upsert=True
    )

This for loop retrieves entries in our data base that do not have rss feed information, retrieves the links through the itunes API and inserts the information into the database until it is stopped. Due to the limitation of twenty API calls per minute this is also a very time consuming process and it is performed by running a script that constantly performes the aforementioned tasks. 

In [ ]:
counter = 0
while True:
    
    query_results = collection.find( { "rss_feed":None } ).limit(20)
    id_list = [item['itunes_id'] for item in query_results]
    
    rss_dict = {}
    for id in id_list:

        itunes_search_link = 'https://itunes.apple.com/lookup?id=' + id.replace('id', '')

        search_request = requests.get(itunes_search_link).json()

        # insert feed into dict
        rss_dict[id] = search_request['results'][0]['feedUrl']
        
    for key in rss_dict:

        result = collection.find_one_and_update(
            {'itunes_id' : key},
            {"$set":
                {'rss_feed': rss_dict[key]}
            },upsert=True
        )
        
    counter = counter + 1
    print(counter)
    time.sleep(60)